In [34]:
# Python libraries:
import os
import sys
import re
from dateutil import parser
# import datetime
from datetime import datetime
from datetime import date
import builtins
import json
import functools
import operator
from itertools import product

# Numpy & Pandas!
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:20,.2f}'.format
pd.options.display.max_columns = None

# Spark!
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *
from pyspark.sql import SparkSession, Row

if 1:
    spark = SparkSession.builder.appName("myapp").getOrCreate()
# else:
#     spark = SparkSession.builder.master("yarn")\
#     .config("spark.executor.instances", "32")\
#     .config("spark.executor.cores", "4")\
#     .config("spark.executor.memory", "4G")\
#     .config("spark.driver.memory", "4G")\
#     .config("spark.executor.memoryOverhead","4G")\
#     .config("spark.yarn.queue","Medium")\
#     .appName("myapp")\
#     .getOrCreate()

sc = spark.sparkContext
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")
spark.conf.set("spark.debug.maxToStringFields","true")

In [37]:
%load_ext autoreload
%autoreload 2
# The autoreload extension is already loaded. To reload it, use:
#  %reload_ext autoreload


# mylib:
my_library = os.path.expanduser('~/.myconfigs')
my_spark = os.path.expanduser('~/spark2_dfanalysis')
sys.path.append(my_library)
sys.path.append(my_spark)


from shared.app_context import *
from builder.DataFrameBuild import *

ctx = ApplicationContext("Dev-Job")

DFB = DataFrameBuild(ctx.spark)

print("%16s  %s" % ("Python Version:",sys.version))
print("%16s  %s" % ("Python Path:",os.path.dirname(sys.executable)))
print("%16s  %s" % ("My Python Libs:",my_library))
print("%16s  %s" % ("My Spark Dir:",my_spark))
print("%16s  %s" % ("My Spark Ctx:",ctx.spark))
# print(ctx.spark)
# print(os.listdir(my_spark))
# print(sys.path)
# print("\n")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
 Python Version:  3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]
    Python Path:  C:\Users\d810216\AppData\Local\conda\conda\envs\my_root
 My Python Libs:  C:\Users\d810216/.myconfigs
   My Spark Dir:  C:\Users\d810216/spark2_dfanalysis
   My Spark Ctx:  <pyspark.sql.session.SparkSession object at 0x000001DD4930CBA8>


# Read / Write / Partition:

In [38]:
num = 500
df4 = DFB.arrays_to_dataframe(
    [DFB.build_array("string",num=num,width=8),
     DFB.build_array("integer",num=num,nrange=(1,4)),
     DFB.build_array("integer",num=num,nrange=(1,12)),
     DFB.build_array("double",num=num,nrange=(0.0,10000))],
    ['passwords','quarter','month','price'])

In [39]:
df4.show()

+---------+-------+-----+------------------+
|passwords|quarter|month|             price|
+---------+-------+-----+------------------+
| auprqiil|      1|    5| 8963.700671314984|
| pmjjgjqs|      3|   12| 8916.362412707324|
| yuvjuoao|      2|   10|1363.1300286611236|
| lutbwshs|      3|    5|2926.8877601218314|
| vblmehnx|      2|    7| 5338.725672465437|
| hnhktvvh|      1|    5| 1402.226609021242|
| yxichabn|      2|   11|143.78983977050953|
| ukdzezrw|      4|    2|1840.4305958818402|
| bgpbrpkv|      3|    7| 5804.103135444399|
| euwcsjvl|      3|    2| 9721.807529800564|
| fevhlrwz|      2|    5|3338.0483490017778|
| qkvzvizh|      1|    6| 60.95092245858358|
| bqnpadfg|      3|    7| 8871.709304473337|
| gpjssrac|      2|    6|  897.412035453572|
| inodxvkm|      1|    1|7858.5255326129345|
| gacgbloa|      2|   10| 7384.484122680922|
| subrovay|      4|   11| 962.1936108193319|
| srdfgfqr|      1|    5| 6274.546864389551|
| vfyzysdh|      3|   11|4954.3215154449135|
| ytcovjdy